In [75]:
import pathlib, base64, json
from datetime import datetime
import pandas as pd
import numpy as np
from io import BytesIO

tree = {}
for f in pathlib.Path('../data').glob('*.parquet'):
    parts = f.name.split('.')
    if len(parts) != 3: continue
    #with f.open('rb') as fp:
    dt = datetime.strptime(parts[0], '%Y%m%d').date()
    tree.setdefault(dt, {})
    df = pd.read_parquet(f)#base64.encodebytes(fp.read()).decode()
    if parts[1] == 'oi':
        df.insert(0, 'Date', np.datetime64(dt))
    tree[dt][parts[1]] = df

p_tree = {}
p_tree['trades'] = pd.concat((d['trades'] for d in tree.values()), ignore_index=True, axis=0).sort_values(by='date_time')
p_tree['oi'] = pd.concat((d['oi'] for d in tree.values()), ignore_index=True, axis=0).sort_values(by='Date')
p_tree['top'] = pd.concat((d['top'] for d in tree.values()), ignore_index=True, axis=0).sort_values(by='date_time')

for k,v in list(p_tree.items()):
    buffer = BytesIO()
    v.to_parquet(buffer)
    buffer.seek(0)
    p_tree[k] = base64.encodebytes(buffer.read()).decode()

with open('tree.json', 'wt') as f:
    json.dump(p_tree, f)